# Michelle's Data (HeCS_pared_small.npy)

In [2]:
import numpy as np
dat = np.load('./data/HeCS_pared_small.npy')
for n in range(len(dat)):
    print('')
    Ngal = dat['Ngal'][n]
    print('cluster index:', n)
    print('velocity standard deviation:', np.std(dat['vlos'][n][0:Ngal])) 
    print ('cluster M500 from SZ:', dat['MSZ'][n])
    print ('cluster M200c:', dat['Mtot'][n])
    print ('  ')


cluster index: 0
velocity standard deviation: 843.07
cluster M500 from SZ: 2.324e+14
cluster M200c: 2.77e+14
  

cluster index: 1
velocity standard deviation: 747.848
cluster M500 from SZ: 2.681e+14
cluster M200c: 2.92e+14
  

cluster index: 2
velocity standard deviation: 426.237
cluster M500 from SZ: 1.141e+14
cluster M200c: 1.19e+14
  

cluster index: 3
velocity standard deviation: 753.334
cluster M500 from SZ: 3.43e+14
cluster M200c: 2.5e+14
  

cluster index: 4
velocity standard deviation: 690.516
cluster M500 from SZ: 2.338e+14
cluster M200c: 1.77e+14
  

cluster index: 5
velocity standard deviation: 485.47
cluster M500 from SZ: 1.309e+14
cluster M200c: 2.02e+14
  

cluster index: 6
velocity standard deviation: 670.823
cluster M500 from SZ: 1.26e+14
cluster M200c: 1.34e+14
  

cluster index: 7
velocity standard deviation: 706.867
cluster M500 from SZ: 2.8e+14
cluster M200c: 2.26e+14
  

cluster index: 8
velocity standard deviation: 750.748
cluster M500 from SZ: 3.437e+14
cluster 